The task is to aproximate the median house value of each block from the values of the rest of the variables. 

This data set contains information about all the block groups in California from the 1990 Census. http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error

c:\users\florian\.conda\envs\tfdl\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\florian\.conda\envs\tfdl\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\florian\.conda\envs\tfdl\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\florian\.conda\envs\tfdl\lib\site-packages\tensorflow\python\framework

In [2]:
dataset = pd.read_csv('cal_housing_clean.csv')
dataset.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [3]:
dataset.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [4]:
# Declare features and labels
x_data = dataset.drop('medianHouseValue',axis=1)
labels = dataset['medianHouseValue']

# Perform a train test split 70/30
x_train, x_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.30)

In [5]:
# Scale the feature data
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(data=scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(data=scaler.transform(x_test), columns=x_test.columns, index=x_test.index)

In [6]:
# Adding the feature columns to Tensorflow
# Since all values a continuus using numeric_column
housingMedianAge = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')

feat_cols = [housingMedianAge, totalRooms, totalBedrooms, population, households, medianIncome]

In [16]:
# Create the input function for tensorflow
input_func = tf.estimator.inputs.pandas_input_fn(x = x_train, y=y_train, batch_size=5, num_epochs=500, shuffle=True)

# Create DNN Regression Model
dnn_model = tf.estimator.DNNRegressor(hidden_units=[2,2,2], feature_columns=feat_cols)
dnn_model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_model_dir': 'C:\\Users\\Florian\\AppData\\Local\\Temp\\tmpegvk89f1', '_save_checkpoints_secs': 600}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Florian\AppData\Local\Temp\tmpegvk89f1\model.ckpt.
INFO:tensorflow:loss = 49225064000.0, step = 1
INFO:tensorflow:global_step/sec: 581.843
INFO:tensorflow:loss = 298524870000.0, step = 101 (0.172 sec)
INFO:tensorflow:global_step/sec: 711.14
INFO:tensorflow:loss = 191109090000.0, step = 201 (0.141 sec)
INFO:tensorflow:global_step/sec: 800.037
INFO:tensorflow:loss = 53115180000.0, step = 301 (0.125 sec)
INFO:tensorflow:global_step/sec: 581.839
INFO:tensorflow:loss = 141288650000.0, step = 401 (0.172 sec)
INFO:tensorflow:

In [17]:
# Model evaluation
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test, y= y_test, batch_size=len(x_test),shuffle=False)
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2020-02-26-15:21:23
INFO:tensorflow:Restoring parameters from C:\Users\Florian\AppData\Local\Temp\tmpegvk89f1\model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2020-02-26-15:21:23
INFO:tensorflow:Saving dict for global step 5000: average_loss = 34011250000.0, global_step = 5000, loss = 210597660000000.0


{'average_loss': 34011250000.0, 'global_step': 5000, 'loss': 210597660000000.0}

In [18]:
# Prediction
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test, y=None, batch_size=len(x_test), shuffle=False)
pred = dnn_model.predict(predict_input_func) # Returns a generator

predictions = list(pred)

# Grab y prediction values
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

mean_squared_error(y_test, final_preds)**0.5

INFO:tensorflow:Restoring parameters from C:\Users\Florian\AppData\Local\Temp\tmpegvk89f1\model.ckpt-5000


184421.3986307136

In [19]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


#mean is at about 200.000 fix the model to get near that!